Distancias entre textos. El último módulo permitirá, dados dos textos, calcular su distancia semántica. 

Para ello, evalúa diferentes alternativas y justifica la elección final tomada. 
La función se denominará texts_distance(text1: str, text2: str)

In [1]:
import pandas as pd

reddit_df = pd.read_csv('processed_dataset.csv', encoding='UTF-8', sep=';', low_memory=False)

Creación de un .txt para el manejo de los embeddings con gensim.

In [5]:
post_corpus = reddit_df['clean_post'].to_list()

with open('clean_post_corpus.txt', 'w', encoding='UTF-8') as f:
    for post in post_corpus:
        f.write(str(post) + '\n')

### Cosine similarity con TF-IDF:

- Ideal para comparar documentos de diferentes longitudes
- Considera la importancia relativa de las palabras
- Computacionalmente eficiente
- Resultados fáciles de interpretar (0-1)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def texts_distance_tfidf(text1: str, text2: str) -> float:
    '''
    Calcula la distancia entre dos textos usando TfIdf y cosine_similarity
    '''
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return 1 - cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]


En este ejemplo, se usó la similitud de coseno como base para medir la distancia. Si dos palabras tienen significados muy similares, su similitud de coseno será cercana a 1, y en consecuencia su distancia (1 - similitud) será cercana a 0.

### Distancia Jaccard

In [ ]:
def jaccard_distance(text1: str, text2: str) -> float:
    '''
    Distancia Jaccard entre dos textos. Simple y rápido, aunque solo
    considera presencia o ausencia de palabras.
    '''
    tokens1 = set(text1.split())
    tokens2 = set(tokens2.split())
    intersection = len(tokens1 & tokens2)
    union = len(tokens1 | tokens2)
    return 1 - (intersection / union if union != 0 else 0)

### **Cambio de entorno virtual**, uso de gensim

### Distancia basada en Word2Vec

La "Word2Vec distance" se basa en la representación vectorial de las palabras (embeddings).

Word2Vec es un modelo que asigna un vector a cada palabra de manera que palabras similares semánticamente estén cerca en el espacio vectorial.
Para medir la similitud (o distancia) entre dos palabras, se calcula por lo general la similitud de coseno entre sus respectivos embeddings.
A diferencia de la frecuencia, esta medida tiene en cuenta el contexto semántico de las palabras y no solo la cuenta de apariciones.

La dificultad del manejo de gensim en paralelo con pandas radica en la diferencia de versiones de NumPy que cada librería utiliza (más atrasada en el caso de gensim).

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import numpy as np
from numpy.linalg import norm

# Entrenamiento del modelo en el corpus .txt

def train_word2vec_model(corpus_path, vector_size=100, min_count=5, workers=4, epochs=10):
    """
    Entrena un modelo Word2Vec usando un corpus de texto línea por línea.
    - corpus_path: ruta al archivo .txt con un post/token por línea (o ya tokenizado).
    - vector_size: dimensión de los embeddings.
    - min_count: ignora palabras con frecuencia < min_count.
    - workers: núm. de CPUs o threads a usar.
    - epochs: número de pasadas completas sobre el corpus.
    
    Retorna el modelo entrenado.
    """
    print("Cargando corpus y entrenando Word2Vec...")
    sentences = LineSentence(corpus_path)
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        min_count=min_count,
        workers=workers,
        epochs=epochs
    )
    print("Entrenamiento finalizado.")
    return model

# Función para obtener vector promedio de un texto

def get_text_embedding(model, tokenized_text):
    """
    Dado un modelo Word2Vec y una lista de tokens,
    retorna el promedio de los vectores de cada token.
    Si no se reconoce ningún token, devuelve un vector de ceros.
    """
    vectors = []
    for token in tokenized_text:
        # Verificamos que el token esté en el vocabulario
        if token in model.wv:
            vectors.append(model.wv[token])
    if not vectors:
        return np.zeros(model.vector_size)
    # Media de los vectores
    return np.mean(vectors, axis=0)


# Función para calcular la distancia entre dos textos

def word2vec_distance(text1_tokens, text2_tokens, model):
    """
    Calcula la similitud y la distancia (1 - similitud_coseno)
    entre dos textos tokenizados.
    
    - text1_tokens: lista de tokens del texto 1.
    - text2_tokens: lista de tokens del texto 2.
    - model: modelo Word2Vec ya entrenado.
    
    Retorna la similitud de coseno y la distancia.
    """
    text1_vec = get_text_embedding(model, text1_tokens)
    text2_vec = get_text_embedding(model, text2_tokens)
    
    # Evitar divisiones por cero si algún vector es nulo
    if not np.any(text1_vec) or not np.any(text2_vec):
        return 0.0, 1.0  # Similitud cero, distancia = 1
    
    # Similitud de coseno = (A·B) / (||A|| * ||B||)
    cos_sim = np.dot(text1_vec, text2_vec) / (norm(text1_vec) * norm(text2_vec))
    distance = 1 - cos_sim
    
    return cos_sim, distance